In [ ]:
import pandas as pd
import requests
import urllib3

import login as login

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

auth_url = 'https://www.strava.com/oauth/token'
activities_url = 'https://www.strava.com/api/v3/athlete/activities'
gear_url = 'https://www.strava.com/api/v3/gear/{id}'

payload = {
    'client_id': f'{login.client_id}',
    'client_secret': f'{login.client_secret}',
    'refresh_token': f'{login.refresh_token}',
    'grant_type': 'refresh_token',
    'f': 'json'
}

res = requests.post(auth_url, data=payload, verify=False)
access_token = res.json()['access_token']

header = {'Authorization': 'Bearer ' + access_token}

# Strava API only allows 200 results per page. This function loops thorugh until all results are collected
def get_activities_data():
    '''This function gets all activities data from Strava API'''
    # set value of page to start at page 1
    page = 1
    # create an empty list to store all data
    data = []
    # set new_results to True to start the loop
    new_results = True
    while new_results:
        # requests one page at a time (200 results)
        get_activities = requests.get(activities_url, headers=header, params={'per_page': 200, 'page': page}).json()
        # feeback
        print(f"Fetching page {page}")
        print(f"Number of activities fetched: {len(get_activities)}")
        # if there are no results, the loop will stop
        new_results = get_activities
        # add the results to the data list
        data.extend(get_activities)
        # increment the page number
        page += 1
    return pd.json_normalize(data)
        
# get all activities data
activities = get_activities_data()

# convert meters to miles
activities.distance = activities.distance / 1609.34
# convert to mph
activities.average_speed = activities.average_speed * 2.23694
activities.max_speed = activities.max_speed * 2.23694
# convert to feet
activities.total_elevation_gain = activities.total_elevation_gain * 3.28084
activities.elev_high = activities.elev_high * 3.28084
activities.elev_low = activities.elev_low * 3.28084

activities_df = pd.DataFrame(activities)

# get distinct gear id's
gear_list = activities_df['gear_id'].unique()

gear_list = gear_list[~pd.isnull(gear_list)]

def get_gear_data(gear_list):
    '''This fuunction gets gear data from Strava API
    
    Args:
        gear_list (array): List of distinct gear ids
        
        Returns:
            data (JSON): JSON data of gear
        '''
    # create empty list to store gear data
    data = []
    # loop through gear_list and get gear data
    for gear_id in gear_list:
        get_gear = requests.get(gear_url.format(id=gear_id), headers=header).json()
        data.append(get_gear)
    return pd.json_normalize(data)

# get all geat data
gear = get_gear_data(gear_list)

# convert meters to miles
gear.distance = gear.distance / 1609.34

gear = gear.drop(columns=['converted_distance'])

# create base dataframe joining activity and gear data
df = pd.merge(activities_df, gear, how='left', left_on='gear_id', right_on='id', suffixes=('_activity', '_gear')).drop(columns='id_gear')

Fetching page 1
Number of activities fetched: 200
Fetching page 2
Number of activities fetched: 200
Fetching page 3
Number of activities fetched: 200
Fetching page 4
Number of activities fetched: 12
Fetching page 5
Number of activities fetched: 0


In [ ]:
# TODO convert moving_time and elapsed time to H% M% S% format
# TODO convert start_date and start_date_local to datetime

In [68]:
df

,resource_state_activity,name_activity,distance_activity,moving_time,elapsed_time,total_elevation_gain,type,sport_type,workout_type,id_activity,...,nickname,resource_state_gear,retired,distance_gear,brand_name,model_name,description,notification_distance,frame_type,weight
0,2,Afternoon Run,4.007543,40.066667,43.566667,164.698168,Run,Run,NaN,13901597211,...,None,3.0,False,291.8159,Merrell,Vapor Glove 5,None,250.0,NaN,NaN
1,2,Afternoon Run,4.061851,38.866667,38.933333,177.165360,Run,Run,NaN,13884288515,...,None,3.0,False,291.8159,Merrell,Vapor Glove 5,None,250.0,NaN,NaN
2,2,Evening Run,2.363764,21.300000,21.300000,147.637800,Run,Run,NaN,13866802472,...,None,3.0,False,291.8159,Merrell,Vapor Glove 5,None,250.0,NaN,NaN
3,2,Afternoon Run,7.091292,69.250000,69.533333,320.209984,Run,Run,NaN,13838427132,...,None,3.0,False,291.8159,Merrell,Vapor Glove 5,None,250.0,NaN,NaN
4,2,Afternoon Run,2.661588,25.166667,25.233333,192.913392,Run,Run,NaN,13819421295,...,None,3.0,False,291.8159,Merrell,Vapor Glove 5,None,250.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
607,2,Afternoon Run,0.977544,8.533333,8.800000,54.790028,Run,Run,NaN,3903866794,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
608,2,Evening Run,1.007618,10.883333,10.933333,201.115492,Run,Run,NaN,3903866817,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
609,2,Evening Run,1.019735,9.416667,9.416667,51.509188,Run,Run,NaN,3903866790,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
610,2,Morning Walk,1.256602,22.366667,22.366667,30.183728,Walk,Walk,NaN,3391765082,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
